In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
books_with_embbeding = pd.read_csv("../data/books_with_embedding.csv")
books_with_embbeding.head(2)

,title,price,description,bookURL,ImageURL,text,embedding
0,Tips & Trick Menguasai Excel 2021,0,Fungsi Excel merupakan rumus siap pakai yang ...,https://www.gramedia.com/products/tips-trick-m...,https://cdn.gramedia.com/uploads/picture_meta/...,Tips & Trick Menguasai Excel 2021 Fungsi Exce...,"[0.021354165, 0.0145276915, -0.062132616, 0.03..."
1,Kupas Tuntas Microsoft Word 2021,0,Program Microsoft Word 2021 adalah software p...,https://www.gramedia.com/products/kupas-tuntas...,https://cdn.gramedia.com/uploads/picture_meta/...,Kupas Tuntas Microsoft Word 2021 Program Micr...,"[0.006129172, 0.035873156, -0.054442372, -0.03..."


In [6]:
books_with_embbeding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1877 entries, 0 to 1876
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1877 non-null   object
 1   price        1877 non-null   int64 
 2   description  1877 non-null   object
 3   bookURL      1877 non-null   object
 4   ImageURL     1877 non-null   object
 5   text         1877 non-null   object
 6   embedding    1877 non-null   object
dtypes: int64(1), object(6)
memory usage: 102.8+ KB


In [15]:
# Preprocessing: Convert to numpy array
import ast

books_with_embbeding["embedding"] = books_with_embbeding["embedding"].apply(
    lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else np.array(x)
) # Parsing string menjadi list angka (float)

# Calculate Cosine Similarity

### Formula

Cosine similarity antara dua vektor $ \mathbf{A} $ dan $ \mathbf{B} $ didefinisikan sebagai:

$$
\text{cosine\_similarity}(\mathbf{A}, \mathbf{B}) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \cdot \|\mathbf{B}\|}
$$

Untuk kasus membandingkan satu vektor $ \mathbf{A} $ dengan seluruh matriks embedding $ \mathbf{M} $, formulanya menjadi:

$$
\text{cos\_sim} = \frac{\mathbf{A} \cdot \mathbf{M}^T}{\|\mathbf{A}\| \cdot \|\mathbf{M}\|_{\text{row-wise}}}
$$

Dengan:

- $ \mathbf{A} \cdot \mathbf{M}^T $: Dot product antara vektor $ \mathbf{A} $ dan setiap baris dari matriks $ \mathbf{M} $
- $ \|\mathbf{A}\| $: Norma (panjang) dari vektor $ \mathbf{A} $
- $ \|\mathbf{M}\|_{\text{row-wise}} $: Norma masing-masing baris (vektor) dalam matriks $ \mathbf{M} $

In [33]:
# Convert seluruh embedding menjadi array 2D
embedding_matrix = np.stack(books_with_embbeding["embedding"].values)


# Menghitung similarity antar buku menggukana libray scikit-learn
def calc_cos_simil(idx_book: int, top_k=5) -> pd.DataFrame:
    cos_sim = cosine_similarity(embedding_matrix[idx_book].reshape(1, -1), embedding_matrix)
    df = pd.DataFrame({
        "title": books_with_embbeding["title"],
        "score": cos_sim.flatten()
    })

    # Hapus dirinya sendiri (buku dengan idx_book)
    df = df.drop(index=idx_book)

    # Sort berdasarkan similarity tertinggi
    df = df.sort_values(by="score", ascending=False)
    return df.head(top_k)

calc_cos_simil(50)

,title,score
418,Kolaborasi Macro Excel dan Access untuk Membua...,0.977701
781,Belajar Mengotomisasi Tugas Sehari-hari dengan...,0.856957
91,Belajar Sendiri VBA Macro Excel untuk Pemula,0.842986
604,Lebih Mudah Mengelola Database dengan Macro Excel,0.839685
571,Membuat Aplikasi Penggajian dengan VBA Macro E...,0.834954


In [ ]:
# Menghitung similarity antar buku from scratch
def calc_cos_simil_from_scratch(idx_book: int, top_n=5) -> pd.DataFrame:
    cos_sim = np.dot(embedding_matrix[idx_book], embedding_matrix.T) / (
        np.linalg.norm(embedding_matrix[idx_book]) * np.linalg.norm(embedding_matrix, axis=1)
    )
    df = pd.DataFrame({
        "title": books_with_embbeding["title"],
        "score": cos_sim
    })
    # Hapus dirinya sendiri (buku dengan idx_book)
    df = df.drop(index=idx_book)
    # Sort berdasarkan similarity tertinggi
    df = df.sort_values(by="score", ascending=False)
    return df.head(top_n)
calc_cos_simil_from_scratch(50)

,title,score
418,Kolaborasi Macro Excel dan Access untuk Membua...,0.977701
781,Belajar Mengotomisasi Tugas Sehari-hari dengan...,0.856957
91,Belajar Sendiri VBA Macro Excel untuk Pemula,0.842986
604,Lebih Mudah Mengelola Database dengan Macro Excel,0.839685
571,Membuat Aplikasi Penggajian dengan VBA Macro E...,0.834954
